# ch05. 유사성과 모호성

- 중의성 문제; 외부 형태 - 내부 의미
- 동형어 homonym
- 다의어 polysemy
- 단어 중의성 해소 WSD; word-sense disambiguation
- 동의어 synonym - 동의어 집합 synset
- 상위어 hypernym - 하위어 hyponym

## One-hot encoding

- 보통 3만~10만 차원 ; sparse vector
- 벡터 간 연산에서 결과값이 0이 되는, 직교orthogonal하는 경우가 많아짐
- 차원의 저주

## Thesaurus

노드 간 거리를 통해 단어간 유사도 측정

- WordNet (Eng) : 단어에 대한 상위어-하위어 정보 구축, Directed Acyclic Graph 유향 비순환 그래프; 트리 구조 아님, NLTK에 포함
- KorLex : http://korlex.pusan.ac.kr/
- KWN : http://wordnet.kaist.ac.kr/




In [1]:
!pip install nltk

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from nltk.corpus import wordnet as wn

def hypernyms(word) : 
    current_node = wn.synsets(word)[0]
    yield current_node

    while True :
        try:
            current_node = current_node.hypernyms()[0]
            yield current_node
        except IndexError :
            break

In [11]:
hypernyms('policeman')

<generator object hypernyms at 0x7fd94726bd58>

In [12]:
type(hypernyms('policeman'))

generator

In [13]:
list(hypernyms('policeman'))

[Synset('policeman.n.01'),
 Synset('lawman.n.01'),
 Synset('defender.n.01'),
 Synset('preserver.n.03'),
 Synset('person.n.01'),
 Synset('causal_agent.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

In [15]:
list(hypernyms('policeman'))[0]

Synset('policeman.n.01')

## Feature Vector

- 의미가 비슷하다면, 쓰임새가 비슷할 것
- 쓰임새가 비슷하면, 비슷한 문장 안에서 비슷한 역할로 사용될 것
- 함께 나타나는 단어들이 유사할 것

## TF-IDF

- TF ; term frequency, 단어의 문서 내 출현 횟수
- IDF ; inverse document frequency, 단어가 출현한 문서 수의 역수
- 특정 문서에만 잘나타나는 단어 - 중요한 역할 차지하는 단어 확인
- 'the'와 같이 일반적으로 나타나는 단어 제거



In [0]:
import pandas as pd

def get_term_frequency (document, word_dict = None) :
    if word_dict is None : 
        word_dict = {}
    words = document.split()

    for w in words : 
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])
    
    return pd.Series(word_dict).sort_values(ascending=False)

In [0]:
def get_document_frequency(documents) :
    dicts = []
    vocab = set([])
    df = {}

    for d in documents :
        tf = get_term_frequency(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())

    for v in list(vocab) :
        df[v] = 0
        for dict_d in dicts :
            if dict_d.get(v) is not None :
                df[v] += 1

    return pd.Series(df).sort_values(ascending = False)

***TF-IDF 구하는 최종 함수***

In [0]:
def get_tfidf(docs) :
    vocab = {}
    tfs = []
    for d in docs : 
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]
    df = get_document_frequency(docs)

    from operator import itemgetter
    import numpy as np

    stats = []
    for word, freq in vocab.items() :
        tfidfs = []
        for idx in range(len(docs)) :
            if tfs[idx].get(word) is not None :
                tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word]) ]
            else : tfidfs += [0]
        stats.append((word, freq, *tfidfs, max(tfidfs)))
    return pd.DataFrame(stats, columns = ('word', 'frequency', 'doc1', 'doc2', 'doc3', 'max')).sort_values('max', ascending = False)

In [24]:
doc1 = '''
지능 지수 라는 말 들 어 보 셨 을 겁니다 . 여러분 의 지성 을 일컫 는 말 이 죠 . 그런데 심리 지수 란 건 뭘까요 ? 사람 들 이 특정 한 식 으로 행동 하 는 이유 에 대해 여러분 은 얼마나 알 고 계시 나요 ? 또 타인 이나 심지어 여러분 의 행동 을 예측 하 는 일 은 얼마나 잘 하 시 나요 ? 또 , 심리학 에 대해 갖춘 지식 중 에서 어느 정도 나 잘못 된 것 일까요 ? 심리학 에 관한 열 가지 신화 를 통해 잘못 된 것 들 을 알아보 도록 하 죠 . 여러분 은 한 번 쯤 들 어 보 셨 을 법 한 것 은 자신 들 의 심리학 에 대해 고려 할 때 , 거의 항상 남자 는 화성 에서 왔 고 , 여자 는 금성 에서 온 것 같 다고 합니다 . 하지만 실제로 남자 와 여자 는 얼마나 다른 걸까요 ? 이 를 알아보 기 위해 , 일단 남녀 사이 에 확실 하 게 차이 나 는 것 을 살펴보 고 심리학 적 인 성별 간 의 차이점 을 동일 한 척도 상 에서 대비 해 보 도록 하 겠 습니다 . 남자 와 여자 간 에 실제로 차이 나 는 능력 중 하나 는 그 들 이 공 을 얼마나 멀리 던질 수 있 느냐 하 는 것 입니다 . 여기 남자 들 의 데 이타 를 보 시 면 , 정상 분포 곡선 이 라는 걸 볼 수 있 습니다 . 남자 들 소수 는 정말 멀리 던지 고 , 남자 들 소수 는 멀리 던지 지 못하 지만 , 남자 들 대부분 은 평균 적 인 거리 를 던졌 습니다 . 여자 들 도 역시 비슷 한 분포 상태 를 보입니다 만 사실 남녀 사이 엔 커다란 차이 가 있 습니다 . 사실 , 평균 수준 의 남자 라면 모든 여성 중 대략 98 % 보다 더 멀리 던질 수 있 거든요 . 이 와 동일 하 게 표준 화 된 척도 상 에서 심리학 에서 말 하 는 성별 간 의 차이 를 살펴 봅시다 . 심리학자 라는 여러분 에게 말 하 길 남자 들 의 공간 지각 능력 이 여자 들 보다 뛰어나 다고 할 겁니다 . 예 를 들 어 , 지도 읽 는 능력 같 은 건데 , 맞 는 말 입니다 . 하지만 그 차이 의 정도 를 살펴봅시다 . 아주 작 죠 . 두 선 이 너무 근접 해서 거의 겹칠 정도 입니다 .
'''

doc2 = '''
최상 의 제시 유형 은 학습 자 에 좌우 되 는 것 이 아니 라 학습 해야 할 내용 에 따라 좌우 됩니다 . 예 를 들 어 여러분 이 운전 하 기 를 배울 때 실제로 몸 으로 체감 하 는 경험 없이 누군가 가 어떻게 할 지 이야기 하 는 것 을 듣 는 것 만 으로 배울 수 있 습니까 ? 연립 방정식 을 풀 어야 하 는데 종이 에 쓰 지 않 고 머리 속 에서 말 하 는 것 으로 풀 수 가 있 을까요 ? 또는 만일 여러분 이 체감 형식 의 학습 자 유형 이 라면 , 건축학 시험 을 해석 적 춤 을 이용 하 여 수정 할 수 있 을까요 ? 아니 죠 ! 배워야 할 내용 을 제시 된 유형 에 맞추 어야 합니다 , 당신 에게 맞추 는 게 아니 라요 . 여러분 들 상당수 가 " A " 급 의 우등 생 이 라는 걸 아 는데 , 조만간 중등 학력 인증 시험 ( GCSE ) 결과 를 받 게 되 시 겠 네요 . 그런데 , 만일 , 여러분 들 이 희망 했 던 성적 을 받 지 못하 게 된다 해도 여러분 들 의 학습 방식 을 탓 해서 는 안 되 는 겁니다 . 여러분 이 비난 할 수 있 는 한 가지 는 바로 유전자 입니다 . 이건 최근 에 런던 대학교 ( UCL ) 에서 수행 했 던 연구 결과 는 여러 학생 들 과 그 들 의 중등 학력 인증 시험 결과 사이 의 차이 중 58 % 는 유전 적 인 요인 으로 좁혀졌 습니다 . 매우 정밀 한 수치 처럼 들립니다 . 그러면 어떻게 알 수 있 을까요 ? 유전 적 요인 과 환경 적 요인 의 상대 적 기여 도 를 알 고 싶 을 때 우리 가 사용 할 수 있 는 방식 은 바로 쌍둥이 연구 입니다 . 일 란 성 쌍생아 의 경우 환경 적 요인 과 유전 적 요인 모두 를 100 % 똑같이 공유 하 게 되 지만 이란 성 쌍생아 의 경우 는 100 % 동일 한 환경 을 공유 하 지만 유전자 의 경우 여타 의 형제자매 들 처럼 50 % 만 공유 하 게 됩니다 . 따라서 일 란 성 쌍둥이 와 이란 성 쌍둥이 사이 의 인증 시험 결과 가 얼마나 비슷 한지 비교 해 보 고 여기 에 약간 의 수학 적 계산 을 더하 게 되 면 그 수행 능력 의 차이 중 어느 정도 가 환경 적 요인 의 탓 이 고 어느 정도 가 유전자 탓 인지 를 알 수 있 게 됩니다 .
'''

doc3 = '''
그러나 이 이야기 는 세 가지 이유 로 인해 신화 입니다 . 첫째 , 가장 중요 한 건 실험실 가운 은 흰색 이 아니 라 회색 이 었 다 라는 점 이 죠 . 둘째 , 참 여자 들 은 실험 하 기 전 에 와 참여 자 들 이 걱정 을 표현 할 때 마다 상기 시키 는 말 을 들 었 는데 , 전기 충격 이 고통 스럽 기 는 하 지만 , 치명 적 이 지 는 않 으며 실제로 영구 적 인 손상 을 남기 는 일 은 없 을 거 라는 것 이 었 습니다 . 셋째 , 참 여자 들 은 단지 가운 을 입 은 사람 이 시켜 전기 충격 을 주지 는 않 았 죠 . 실험 이 끝나 고 그 들 의 인터뷰 를 했 을 때 모든 참여 자 들 은 강한 신념 을 밝혔 는데 , ' 학습 과 처벌 ' 연구 가 과학 적 으로 가치 있 는 목적 을 수행 했 기 때문 에 비록 동료 참여 자 들 에게 가해진 순간 적 인 불편 함 에 반해서 과학 을 위해서 오래 남 을 성과 를 얻 을 것 이 라고 말 이 죠 . 그러 다 보 니 제 가 이야기 를 한 지 벌써 12 분 이 되 었 습니다 . 여러분 들 중 에 는 아마 거기 앉 아서 제 이야기 를 들으시는 동안 저 의 말투 와 몸짓 을 분석 하 면서 제 가 말 하 는 어떤 것 을 인지 해야 할까 해결 하 려고 하 셨 을 겁니다 , 제 가 진실 을 이야기 하 는 지 , 또는 거짓말 을 하 고 있 는 것 인지 말 이 죠 . 만일 그러 셨 다면 , 아마 지금 쯤 완전히 실패 하 셨 을 겁니다 . 왜냐하면 우리 모두 가 사람 이 말 하 는 패턴 과 몸짓 으로 도 거짓말 여부 를 알아내 는 것 이 가능 하 다고 생각 하 지만 , 오랜 세월 수백 회 에 걸쳐 행해진 실제 심리 검사 의 결과 를 보 면 우리 들 모두 는 , 심지어 경찰관 이나 탐정 들 을 포함 해서 도 기본 적 으로 몸짓 과 언어 적 패턴 으로 거짓말 을 탐지 하 는 것 은 운 에 맞 길 수 밖 에 는 없 는 것 입니다 . 흥미 롭 게 도 한 가지 예외 가 있 는데요 : 실종 된 친척 을 찾 아 달 라고 호소 하 는 TV 홍보 입니다 .
'''

display (get_tfidf([doc1, doc2, doc3]).head(10))
display (get_tfidf([doc1, doc2, doc3]).tail(10))

,word,frequency,doc1,doc2,doc3,max
23,남자,9,9.887511,0.000000,0.000000,9.887511
37,요인,6,0.000000,6.591674,0.000000,6.591674
51,심리학,5,5.493061,0.000000,0.000000,5.493061
57,었,4,0.000000,0.000000,4.394449,4.394449
63,제,4,0.000000,0.000000,4.394449,4.394449
70,시험,4,0.000000,4.394449,0.000000,4.394449
69,멀리,4,4.394449,0.000000,0.000000,4.394449
68,환경,4,0.000000,4.394449,0.000000,4.394449
64,성,4,0.000000,4.394449,0.000000,4.394449
98,인증,3,0.000000,3.295837,0.000000,3.295837


,word,frequency,doc1,doc2,doc3,max
45,된,5,0.0,0.0,0.0,0.0
44,그,5,0.0,0.0,0.0,0.0
42,지만,5,0.0,0.0,0.0,0.0
39,때,5,0.0,0.0,0.0,0.0
38,와,6,0.0,0.0,0.0,0.0
36,라는,6,0.0,0.0,0.0,0.0
33,중,6,0.0,0.0,0.0,0.0
30,습니다,7,0.0,0.0,0.0,0.0
29,보,7,0.0,0.0,0.0,0.0
0,는,47,0.0,0.0,0.0,0.0


### TF Matrix

문서별 단어 출현 횟수

In [0]:
def get_tf(docs) :
    vocab = {}
    tfs = []
    for d in docs : 
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]

    from operator import itemgetter
    import numpy as np

    stats= []
    for word, freq in vocab.items():
        tf_v = []
        for idx in range(len(docs)) :
            if tfs[idx].get(word) is not None :
                tf_v += [tfs[idx][word]]
            else : 
                tf_v += [0]
        stats.append((word, freq, *tf_v))
    return pd.DataFrame(stats, columns=('word','frequency','doc1','doc2','doc3')).sort_values('frequency', ascending=False)

In [26]:
get_tf([doc1, doc2, doc3])

,word,frequency,doc1,doc2,doc3
0,는,47,15,14,18
1,을,39,8,10,21
2,.,36,16,10,10
3,하,33,10,9,14
4,이,32,8,8,16
...,...,...,...,...,...
273,스럽,1,0,0,1
274,치명,1,0,0,1
275,으며,1,0,0,1
276,영구,1,0,0,1


In [28]:
tf_df = get_tf([doc1, doc2, doc3])
tf_df[tf_df['word'] == '여러분']

,word,frequency,doc1,doc2,doc3
15,여러분,12,5,6,1


#### Context Window

- windowing ; 함께 나타나는 단어 조사, 동시발생 단어 활용
- t-SNE 를 통해 시각화 ; t-distributed stochastic neighbor embedding

In [0]:
from collections import defaultdict
import pandas as pd

def get_context_counts(lines, w_size = 2) :
    co_dict = defaultdict(int)

    for line in lines : 
        words = line.split()

        for i, w in enumerate(words) :
            for c in words[i - w_size : i + w_size ] :
                if w != c :
                    co_dict[(w, c)] += 1

    return pd.Series(co_dict)

In [0]:
def co_occurrence(co_dict, vocab) :
    data = []
    for word1 in vocab :
        row = []
        for word2 in vocab :
            try:
                count = co_dict[(word1, word2)]
            except KeyError : 
                count = 0
            row.append(count)
        data.append(row)
    return pd.DataFrame(data, index = vocab, columns = vocab)

## Vector Similarity

In [0]:
import torch

### L1 distance

- Manhattan Distance

In [0]:
def get_l1_distance(x1, x2) :
    return ((x1 - x2).abs()).sum()

### L2 distance

- Euclidean Distance

In [0]:
def get_l2_distance (x1, x2) :
    return ((x1 - x2) ** 2).sum() ** 0.5

### infinity Norm


In [0]:
def get_infinity_distance(x1, x2) :
    return ((x1 - x2).abs()).max()

### Cosine Similarity

- 가장 널리 쓰이는 유사도 측정 방법
- 다만 벡터 차원 크기 커질수록 연산량 크게 증가, 차원의 저주




In [0]:
def get_cosine_similarity (x1, x2) :
    return (x1 * x2).sum() / ((x1 ** 2).sum() ** .5 * (x2 ** 2).sum() ** .5)

### Jaccard Similarity

- 두 집합의 교집합 크기를 합집합 크기로 나누는 방식

In [0]:
def get_jaccard_similarity (x1, x2) :
    return torch.stack([x1, x2]).min(dim = 0)[0].sum() / torch.stack([x1, x2]).max(dim = 0)[0].sum()

In [0]:
torch.stack?

위에서 만든 context windown를 dataframe으로 불러와서 각 유사도 측정방법으로 측정

In [0]:
def get_nearest(query:str, dataframe, metric, top_k, ascending=True)
    vector = torch.FloatTensor(dataframe.loc[query].values)
    distances = dataframe.apply(lambda x : metric(vector, torch.FloatTensor(x.values)), axis = 1)
    top_distances = distances.sort_values(ascending = ascending)[:top_k]

    print (', '.join([f"{k} ({v:.1f})" for k, v in top_distances.items()]))

query = '우리'
get_nearest(query, dataframe, get_l1_distance, 30 )
get_nearest(query, dataframe, get_l2_distance, 30 )
get_nearest(query, dataframe, get_infinity_distance, 30 )
get_nearest(query, dataframe, get_cosine_similarity, 30 )
get_nearest(query, dataframe, get_jaccard_similarity, 30 )

## 단어 중의성 해소

### Lesk Algorithm

- 가장 간단한 사전 기반 중의성 해소 방법
- 가정 : 문장 내 같이 등장하는 단어들은 공통 토픽을 공유


In [40]:
from nltk.corpus import wordnet as wn
for ss in wn.synsets('bass') : print (ss, ss.definition())

Synset('bass.n.01') the lowest part of the musical range
Synset('bass.n.02') the lowest part in polyphonic music
Synset('bass.n.03') an adult male singer with the lowest voice
Synset('sea_bass.n.01') the lean flesh of a saltwater fish of the family Serranidae
Synset('freshwater_bass.n.01') any of various North American freshwater fish with lean flesh (especially of the genus Micropterus)
Synset('bass.n.06') the lowest adult male singing voice
Synset('bass.n.07') the member with the lowest range of a family of musical instruments
Synset('bass.n.08') nontechnical name for any of numerous edible marine and freshwater spiny-finned fishes
Synset('bass.s.01') having or denoting a low vocal or instrumental range


In [47]:
def lesk(sentence, word) :
    from nltk.wsd import lesk
    best_synset = lesk( sentence.split(), word)
    print (best_synset, best_synset.definition())

sentence = u'I went fishing last weekend and I got a bass and cooked it'
word = 'bass'
lesk(sentence, word)

Synset('sea_bass.n.01') the lean flesh of a saltwater fish of the family Serranidae


In [48]:
sentence = 'I love the music from the speaker which has strong beat and bass'
word = 'bass'
lesk(sentence, word)

Synset('bass.n.02') the lowest part in polyphonic music


In [49]:
sentence = 'I think the bass is more important than guitar'
word = 'bass'
lesk(sentence, word)

Synset('sea_bass.n.01') the lean flesh of a saltwater fish of the family Serranidae


## Selectional Preference

- 문장 내 주변 단어들에 따라 의미 추정
- 단어-단어 관계가 좀더 특별한 경우를 수치화
- Selectional preference strength : KLD (쿨백-라이블러 발산) 활용, 술어 동사predicative verb 주어졌을 때 목적어 역할의 표제어headword 단어들의 분포
- selectional association : 특정 동사와 좀더 자주 출현하는 클래스의 목적어일수록 높게 나타남
- WordNet 기반 selectional preference 추정
- pseudo word 유사어휘를 통한 selectional preference 평가
- 유사도 기반 selectional preference ; WordNet 기반 추정의 경우 특정 언어에 국한, 신조어 반영 불가, thesaurus에 의존하지 않는 데이터 기반 선택 선호도 추정; corpus에 따라 유사도 추정 대상이 달라지는 coverage 문제





In [0]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [0]:
!pip install konlpy

In [0]:
from konlpy.tag import Mecab, Kkma

In [0]:
def count_seen_headwords(lines, predicate = 'VV', headword = 'NNG') :
    tagger = Kkma()
    seen_dict = {}

    for line in lines : 
        pos_result = tagger.pos(line)
        word_h = None
        word_p = None

        for word, pos in pos_result :
            if pos == predicate or pos[:3] == predicate + '+' :
                word_p = word
                break
            if pos == headword :
                word_h = word
        if word_h is not None and word_p is not None :
            seen_dict[word_p] = [word_h] + ([] if seen_dict.get(word_p) is None else seen_dict[word_p])
    
    return seen_dict


In [0]:
def get_selectional_association(predicate, headword, lines, dataframe, metric) :
    v1 = torch.FloatTensor(dataframe.loc[headword].values)

    # seen_headwords : 위의 count_seen_headwords()로 return 한 값
    seens = seen_headwords[predicate]

    total = 0
    for seen in seens :
        try :
            v2 = torch.FloatTensor(dataframe.loc[seen].values)
            total += metric(v1, v2)
        except :
            pass
    return total

In [0]:
def wsd (predicate, headwords) :
    selectional_associations = []
    for h in headwords :
        selectional_associations += [get_selectional_association(predicate, h, lines, co, get_cosine_similarity)]
    print (selectional_associations)